In [1]:
# !pip install praw
# !pip install requests-cache
# !pip install pymongo

In [2]:
import requests_cache
requests_cache.install_cache('web_cache')

In [3]:
import praw
from pymongo import MongoClient
import json
import pandas as pd
from tqdm.notebook import tqdm
from IPython.display import JSON

In [4]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
raw = db['reddit-raw']

In [5]:
credentials = ""

with open('./reddit_credentials.json') as f:
    credentials = json.load(f)

In [6]:
reddit = praw.Reddit(client_id=credentials['client_id'],
                     client_secret=credentials['client_secret'],
                     password=credentials['password'],
                     user_agent=credentials['user_agent'],
                     username=credentials['username'])

In [7]:
class PRAWJSONEncoder(json.JSONEncoder):
    """Class to encode PRAW objects to JSON."""
    """From https://gist.github.com/jarhill0/6e6495706252d52573950c3820f533b0"""
    def default(self, obj):
        if isinstance(obj, praw.models.base.PRAWBase):
            obj_dict = {}
            for key, value in obj.__dict__.items():
                if not key.startswith('_'):
                    obj_dict[key] = value
            return obj_dict
        else:
            return json.JSONEncoder.default(self, obj)

def digest_submission (submission):
    if not raw.find_one({'id': submission.id}, {'_id': 1}):
        return raw.insert_one(json.loads(json.dumps(submission, cls=PRAWJSONEncoder)))
    else:
        return ''
    
def submissions_iter (r, sort, limit):
    if sort == 'top_all':
        return reddit.subreddit(r).top('all', limit=limit)
    elif sort == 'top_year':
        return reddit.subreddit(r).top('year', limit=limit)
    else:
        return reddit.subreddit(r).new(limit=limit)

def retrieve_submissions (r='dataisugly', sort='top_all', limit=100):
    return (digest_submission(s) for s in tqdm(submissions_iter(r, sort, limit), total=limit))

In [8]:
# crawler_queue = [
#     {'r': 'dataisugly', 'sort': 'top_all', 'limit': 10}
# ]

crawler_queue = [
    {'r': 'dataisugly', 'sort': 'top_all', 'limit': 1000},
    {'r': 'dataisugly', 'sort': 'top_year', 'limit': 1000},
    {'r': 'dataisugly', 'sort': 'new', 'limit': 1000}
]

In [10]:
for c in crawler_queue:
    [s for s in retrieve_submissions(r=c['r'], sort=c['sort'], limit=c['limit'])]